In [1]:
from urllib.request import urlopen
from io import BytesIO
from PIL import Image
from imagehash import phash
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.ml.linalg import Vectors

import numpy as np

In [2]:
joined_df = sqlContext.read.parquet("/outputs/idigbio-media-20171112T013207-mercury-usecase.parquet")

In [ ]:
joined_df.count()

In [3]:
from pyspark.ml.linalg import *
from pyspark.ml.linalg import DenseVector, VectorUDT
from pyspark.ml.feature import StringIndexer, StandardScaler

def image_nparray(v):
    try:
        resp = urlopen("http://api.idigbio.org/v2/media?size=webview&filereference=" + v)
        pilImage = Image.open(BytesIO(resp.read())).resize((256,256),Image.LANCZOS)
        array = np.asarray(pilImage).astype(float)
        array = array.ravel()
        return array.tolist()
    except:
        return np.nan

    
joined_df = sqlContext.read.parquet("/outputs/idigbio-media-20171112T013207-mercury-usecase.parquet")
imgnpa_udf = F.udf(image_nparray, T.ArrayType(T.FloatType()))
npdf = joined_df.select("accessuri","contaminated",imgnpa_udf("accessuri"))


to_vectors = F.udf(lambda x: DenseVector(x), VectorUDT())
vector_df = npdf.withColumn("vector_images",to_vectors("image_nparray(accessuri)"))



get_vector_length = F.udf(lambda vec : vec.size, T.IntegerType())
vector_length_df = vector_df.withColumn("vector_length",get_vector_length("vector_images"))
#vector_length_df.describe().show()

equal_df = vector_length_df.filter(F.col("vector_length")==196608)
same_length_df = equal_df.select("vector_images","contaminated")

# Standardizes features by scaling to unit variance and/or removing the mean using column summary statistics on 
# the samples in the training set
# Standardization can improve the convergence rate during the optimization process, and also prevents against 
# features with very large variances exerting an overly large influence during model training.


scaler = StandardScaler(inputCol="vector_images", outputCol="scaled_features", withStd=True, withMean=False)
#fitted_scaler = scaler.fit(same_length_df)
# scaled_df = fitted_scaler.transform(same_length_df)
input_data = np.array(same_length_df.select('vector_images').collect())


Py4JJavaError: An error occurred while calling o97.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 2.0 failed 4 times, most recent failure: Lost task 6.3 in stage 2.0 (TID 35, mesos08.acis.ufl.edu, executor 2): ExecutorLostFailure (executor 2 exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 144986 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1499)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1487)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1486)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1486)
	at org.apache.spark.s

In [4]:
input_data = np.array(same_length_df.select('vector_images').collect())

Py4JJavaError: An error occurred while calling o97.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 2.0 failed 4 times, most recent failure: Lost task 8.3 in stage 2.0 (TID 35, mesos11.acis.ufl.edu, executor 2): ExecutorLostFailure (executor 2 exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 135442 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1499)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1487)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1486)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1486)
	at org.apache.spark.s

In [5]:
same_length_df.printSchema()

root
 |-- vector_images: vector (nullable = true)
 |-- contaminated: integer (nullable = true)



In [ ]:
dd = same_length_df.select('vector_images')
#np.asarray([x[0] for x in dd])

In [7]:
input_images.shape

NameError: name 'input_images' is not defined

In [1]:
x

NameError: name 'x' is not defined

In [12]:
faulty_df.count()

17768

In [13]:
fitted_scaler = scaler.fit(faulty_df)
scaled_df = fitted_scaler.transform(faulty_df)

Py4JJavaError: An error occurred while calling o67.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 11.0 failed 4 times, most recent failure: Lost task 4.3 in stage 11.0 (TID 102, mesos03.acis.ufl.edu, executor 1): ExecutorLostFailure (executor 1 exited caused by one of the running tasks) Reason: Executor finished with state KILLED
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1499)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1487)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1486)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1486)
	at org.apache.spark.scheduler.DAGSchedu

In [5]:
from pyspark.ml.linalg import *

def image_nparray(v):
    try:
        resp = urlopen("http://api.idigbio.org/v2/media?size=webview&filereference=" + v)
        pilImage = Image.open(BytesIO(resp.read())).resize((256,256),Image.LANCZOS)
        array = np.asarray(pilImage).astype(float)
        array = array.ravel()
        return array.tolist()
    except:
        return np.nan

In [6]:
imgnpa_udf = F.udf(image_nparray, T.ArrayType(T.FloatType()))
npdf = joined_df.select("accessuri","contaminated",imgnpa_udf("accessuri"))
npdf.write....

In [6]:
npdf.show()

+--------------------+------------+------------------------+
|           accessuri|contaminated|image_nparray(accessuri)|
+--------------------+------------+------------------------+
|http://collection...|           0|    [1.0, 1.0, 1.0, 1...|
|http://collection...|           1|    [212.0, 212.0, 20...|
|http://collection...|           1|    [45.0, 44.0, 51.0...|
|http://collection...|           0|    [16.0, 16.0, 16.0...|
|http://collection...|           1|    [3.0, 5.0, 5.0, 3...|
|http://collection...|           0|    [19.0, 19.0, 19.0...|
|http://collection...|           1|    [1.0, 1.0, 1.0, 1...|
|http://collection...|           0|    [212.0, 212.0, 20...|
|http://collection...|           0|    [187.0, 189.0, 17...|
|http://collection...|           0|    [10.0, 10.0, 10.0...|
|http://collection...|           0|    [206.0, 208.0, 19...|
|http://collection...|           1|    [2.0, 2.0, 2.0, 3...|
|http://collection...|           0|    [208.0, 208.0, 19...|
|http://collection...|  

In [7]:
from pyspark.ml.linalg import DenseVector, VectorUDT
to_vectors = F.udf(lambda x: DenseVector(x), VectorUDT())
vector_df = npdf.withColumn("vector_images",to_vectors("image_nparray(accessuri)"))
vector_df.show()

+--------------------+------------+------------------------+--------------------+
|           accessuri|contaminated|image_nparray(accessuri)|       vector_images|
+--------------------+------------+------------------------+--------------------+
|http://collection...|           0|    [1.0, 1.0, 1.0, 1...|[1.0,1.0,1.0,1.0,...|
|http://collection...|           1|    [212.0, 212.0, 20...|[212.0,212.0,202....|
|http://collection...|           1|    [45.0, 44.0, 51.0...|[45.0,44.0,51.0,4...|
|http://collection...|           0|    [16.0, 16.0, 16.0...|[16.0,16.0,16.0,1...|
|http://collection...|           1|    [3.0, 5.0, 5.0, 3...|[3.0,5.0,5.0,32.0...|
|http://collection...|           0|    [19.0, 19.0, 19.0...|[19.0,19.0,19.0,1...|
|http://collection...|           1|    [1.0, 1.0, 1.0, 1...|[1.0,1.0,1.0,1.0,...|
|http://collection...|           0|    [212.0, 212.0, 20...|[212.0,212.0,202....|
|http://collection...|           0|    [187.0, 189.0, 17...|[187.0,189.0,177....|
|http://collecti

In [8]:
from pyspark.ml.feature import StringIndexer, StandardScaler

scaler = StandardScaler(inputCol="vector_images", outputCol="scaled_features", withStd=True, withMean=False)

In [13]:
get_vector_length = F.udf(lambda vec : vec.size, T.IntegerType())
vector_length_df = vector_df.withColumn("vector_length",get_vector_length("vector_images"))
vector_length_df.describe().show()


+-------+--------------------+------------------+------------------+
|summary|           accessuri|      contaminated|     vector_length|
+-------+--------------------+------------------+------------------+
|  count|               17769|             17769|             17769|
|   mean|                null|0.4779672463278744|196596.93539310034|
| stddev|                null|0.4995283782560329|1474.9166650088457|
|    min|http://collection...|                 0|                 1|
|    max|http://collection...|                 1|            196608|
+-------+--------------------+------------------+------------------+



In [19]:
vector_length_df = vector_length_df.cache()

In [20]:
vector_length_df.show(10)

KeyboardInterrupt: 

In [21]:
from pyspark.sql.functions import col
vector_length_df.filter(col("vector_length")==1).show(10)

Py4JJavaError: An error occurred while calling o173.showString.
: org.apache.spark.SparkException: Job 12 cancelled 
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1499)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1439)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:1686)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:1669)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:1658)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:630)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2022)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2043)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2062)
	at org.apache.spark.sql.e